# Advent of Code 2019

## Utils

In [47]:
import os.path
from itertools import product
from collections import defaultdict, Counter, deque
import operator

def read_lines(day):
    filename = os.path.join('input', str(day) + '.txt')
    with open(filename) as f:
        return list(map(str.strip, f))

## Day 1: The Tyranny of the Rocket Equation

In [4]:
def fuel(mass):
    return mass // 3 - 2

In [11]:
masses = list(map(int, read_lines(1)))

In [14]:
print('Day 1.1:', sum(map(fuel, masses)))

Day 1.1: 3270338


In [29]:
def total_fuel(mass):
    total = 0
    while mass // 3 > 2:
        mass = mass // 3 - 2
        total += mass
    return total

In [30]:
print('Day 1.2:', sum(map(total_fuel, masses)))

Day 1.2: 4902650


# Day 2: 1202 Program Alarm

In [27]:
program = list(map(int, read_lines(2)[0].rstrip().split(',')))

In [28]:
def run_program(noun, verb, p):
    p = p[:]
    p[1] = noun
    p[2] = verb
    pc = 0
    while p[pc] != 99:
        if p[pc] == 1:
            p[p[pc + 3]] = p[p[pc + 1]] + p[p[pc + 2]]
        elif p[pc] == 2:
            p[p[pc + 3]] = p[p[pc + 1]] * p[p[pc + 2]]
        pc += 4
    return p[0]

In [38]:
print('Day 2.1:', run_program(12, 2, program))

Day 2.1: 3101844


In [48]:
def find_output(p, output):
    for noun, verb in product(range(0, 100), range(0, 100)):
        if run_program(noun, verb, p[:]) == output:
            return noun, verb

In [51]:
noun, verb = find_output(program, 19690720)
print('Day 2.2:', 100 * noun + verb)

Day 2.2: 8478


# Day 3: Crossed Wires

In [56]:
wire_paths = [line.split(',') for line in read_lines(3)]

In [57]:
def walk_path(path):
    
    DIRECTIONS_MOVE = {
        'R' : -1,
        'L' : 1,
        'U' : 1j,
        'D' : -1j
    }
    
    start = 0j
    for segment in path:
        direction = segment[0]
        nsteps = int(segment[1:])
        for _ in range(nsteps):
            start += DIRECTIONS_MOVE[direction]
            yield start

In [58]:
def manhattam(p1, p2):
    diff = p1 - p2
    return abs(diff.imag) + abs(diff.real)

In [59]:
def intersections(path1, path2):
    hits = defaultdict(set)
    for c1 in path1:
        hits[c1].add(1)
    for c2 in path2:
        hits[c2].add(2)
    return [coordinate for coordinate, hit in hits.items() if len(hit) == 2]

In [60]:
closest_intersection = min(manhattam(0j, i) for i in intersections(walk_path(wire_paths[0]), walk_path(wire_paths[1])))

In [61]:
print('Day 3.1:', closest_intersection)

Day 3.1: 806.0


In [69]:
def intersection_steps(path1, path2):
    hits = defaultdict(dict)
    for i, c1 in enumerate(path1):
        if 1 not in hits[c1]:
            hits[c1][1] = i + 1
    for i, c2 in enumerate(path2):
        if 2 not in hits[c2]:
            hits[c2][2] = i + 1
    return [(hit[1], hit[2]) for hit in hits.values() if len(hit) == 2]

In [70]:
closest_delay = min(steps1 + steps2 for steps1, steps2 in intersection_steps(walk_path(wire_paths[0]), walk_path(wire_paths[1])))

In [72]:
print('Day 3.2:', closest_delay)

Day 3.2: 66076


# Day 4: Secure Container

In [1]:
start, end = 359282, 820401

In [2]:
def increasing(str_num):
    for i in range(len(str_num) - 1):
        if str_num[i] > str_num[i + 1]:
            return False
    return True

In [6]:
def consecutives(str_num):
    for i in range(len(str_num) - 1):
        if str_num[i] == str_num[i + 1]:
            return True
    return False

In [10]:
num_candidate_passwords = sum(increasing(str(num)) and consecutives(str(num)) for num in range(start, end + 1))

In [11]:
print('Day 4.1:', num_candidate_passwords)

Day 4.1: 511


In [21]:
def consecutive_pair(str_num):
    c = Counter(str_num)
    return any(count == 2 for digit, count in c.items())

In [25]:
num_candidate_passwords = sum(increasing(str(num)) and consecutive_pair(str(num)) for num in range(start, end + 1))

In [27]:
print('Day 4.2:', num_candidate_passwords)

Day 4.2: 316


# Day 5: Sunny with a Chance of Asteroids

In [80]:
program = list(map(int, read_lines(5)[0].split(',')))

In [81]:
def decode_op(code):
    return code % 100

In [82]:
def load(mode, value, memory):
    return memory[value] if mode == 0 else value

In [83]:
def store(mode, value, memory, content):
    if mode == 0:
        memory[value] = content
    else:
        memory[memory[value]] = content

In [84]:
def decode(op):
    inst = op % 100
    op //= 100
    op1_mode = op % 10
    op //= 10
    op2_mode = op % 10
    op //= 10
    op3_mode = op
    return (inst, op1_mode, op2_mode, op3_mode)

In [85]:
def ops():
    
    def add(code, memory, pc_ref, in_channel, out_channel):
        _, m1, m2, _ = decode(code)
        result = load(m1, memory[pc_ref[0] + 1], memory) + load(m2, memory[pc_ref[0] + 2], memory)
        memory[memory[pc_ref[0] + 3]] = result
        pc_ref[0] += 4
    
    def mull(code, memory, pc_ref, in_channel, out_channel):
        _, m1, m2, _ = decode(code)
        result = load(m1, memory[pc_ref[0] + 1], memory) * load(m2, memory[pc_ref[0] + 2], memory)
        memory[memory[pc_ref[0] + 3]] = result
        pc_ref[0] += 4
    
    def in_p(code, memory, pc_ref, in_channel, out_channel):
        memory[memory[pc_ref[0] + 1]] = in_channel.pop()
        pc_ref[0] += 2
    
    def out_p(code, memory, pc_ref, in_channel, out_channel):
        _, m1, _, _ = decode(code)
        out_channel.append(load(m1, memory[pc_ref[0] + 1], memory))
        pc_ref[0] += 2
        
    def jift(code, memory, pc_ref, in_channel, out_channel):
        _, m1, m2, _ = decode(code)
        a1 = load(m1, memory[pc_ref[0] + 1], memory)
        pc = load(m2, memory[pc_ref[0] + 2], memory)
        if a1 != 0:
            pc_ref[0] = pc
        else:
            pc_ref[0] += 3
        
    def jiff(code, memory, pc_ref, in_channel, out_channel):
        _, m1, m2, _ = decode(code)
        a1 = load(m1, memory[pc_ref[0] + 1], memory)
        pc = load(m2, memory[pc_ref[0] + 2], memory)
        if a1 == 0:
            pc_ref[0] = pc
        else:
            pc_ref[0] += 3
    
    def less(code, memory, pc_ref, in_channel, out_channel):
        _, m1, m2, m3 = decode(code)
        a1 = load(m1, memory[pc_ref[0] + 1], memory)
        a2 = load(m2, memory[pc_ref[0] + 2], memory)
        if a1 < a2:
            store(m3, memory[pc_ref[0] + 3], memory, 1)
        else:
            store(m3, memory[pc_ref[0] + 3], memory, 0)
        pc_ref[0] += 4
    
    def eq(code, memory, pc_ref, in_channel, out_channel):
        _, m1, m2, m3 = decode(code)
        a1 = load(m1, memory[pc_ref[0] + 1], memory)
        a2 = load(m2, memory[pc_ref[0] + 2], memory)
        if a1 == a2:
            store(m3, memory[pc_ref[0] + 3], memory, 1)
        else:
            store(m3, memory[pc_ref[0] + 3], memory, 0)
        pc_ref[0] += 4
    
    return [None, add, mull, in_p, out_p, jift, jiff, less, eq]

In [86]:
def run_program(ops, program, in_c, out_c):
    memory = program[:]
    pc_ref = [0]
    while True:
        op = decode_op(memory[pc_ref[0]])
        if op == 99:
            break
            
        ops[op](memory[pc_ref[0]], memory, pc_ref, in_c, out_c)

In [87]:
inc, outc = [1], []

In [88]:
run_program(ops(), program, inc, outc)

In [90]:
print('Day 5.1', outc[-1])

Day 5.1 13294380


In [91]:
inc, outc = [5], []

In [92]:
run_program(ops(), program, inc, outc)

In [93]:
print('Day 5.2', outc[-1])

Day 5.2 11460760


# Day 6: Universal Orbit Map

In [29]:
orbits = [list(reversed(line.split(')'))) for line in read_lines(6)]

In [30]:
graph = defaultdict(list)

In [31]:
for orbiter, orbited in orbits:
    graph[orbiter].append(orbited)

In [32]:
def total_orbits(graph):
    
    orbit_cache = dict()
    
    def compute_orbits(key):
        if key not in orbit_cache:
            orbit_cache[key] = sum(compute_orbits(nxt) + 1 for nxt in graph[key])
        return orbit_cache[key]
    
    for key in list(graph):
        if key not in orbit_cache:
            orbit_cache[key] = compute_orbits(key)
    
    return sum(orbit_cache.values())

In [34]:
print('Day 6.1', total_orbits(graph))

Day 6.1 402879


In [35]:
bi_graph = defaultdict(list)

In [37]:
for orbiter, orbited in orbits:
    bi_graph[orbiter].append(orbited)
    bi_graph[orbited].append(orbiter)

In [44]:
start = bi_graph['SAN'][0]

In [45]:
end = bi_graph['YOU'][0]

In [53]:
def bfs(graph, start, end):
    visited = set()
    pending = deque([(start, 0)])
    while len(pending) > 0:
        key, level = pending.popleft()
        
        if key == end:
            return level
        
        if key not in visited:
            visited.add(key)
            for nxt in graph[key]:
                pending.append((nxt, level + 1))

    return -1

In [55]:
print('Day 6.2', bfs(bi_graph, start, end))

Day 6.2 484
